### Parse `Husky`

In [16]:
import mujoco
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('../../')
from utils.mujoco_parser import MuJoCoParserClass
from utils.util import r2rpy
np.set_printoptions(precision=2,suppress=True,linewidth=100)
plt.rc('xtick',labelsize=6); plt.rc('ytick',labelsize=6)
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
print ("MuJoCo version:[%s]"%(mujoco.__version__))

MuJoCo version:[2.3.7]


### Parse `scene_husky_w_ur5_rg2_d435i.xml`

In [17]:
xml_path = '../../asset/husky/scene_husky_w_ur5_rg2_d435i.xml'
env = MuJoCoParserClass(name='Husky',rel_xml_path=xml_path,VERBOSE=True)
print ("[%s] parsed."%(env.name))

## spawn arbitrary objects
obj_cylinder_names = [body_name for body_name in env.body_names
             if body_name is not None and (body_name.startswith("obj_cylinder"))]

n_cylinder_obj = len(obj_cylinder_names)

# Place objects
env.place_objects(n_obj=n_cylinder_obj, obj_names=obj_cylinder_names, x_range=[-1.5, -2.0], y_range=[-0.5, 0.5], z_range=[0.2, 0.3], COLORS=True, VERBOSE=True)

dt:[0.0020] HZ:[500]
n_dof (=nv):[70]
n_geom:[67]
geom_names:['floor', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
n_body:[32]
body_names:['world', 'base_husky', 'front_left_wheel_link', 'front_right_wheel_link', 'rear_left_wheel_link', 'rear_right_wheel_link', 'base_ur5e', 'shoulder_link', 'upper_arm_link', 'forearm_link', 'wrist_1_link', 'wrist_2_link', 'wrist_3_link', 'tcp_link', 'camera_mount', 'd435i', 'rg2_gripper_base_link', 'camera_center', 'rg2_gripper_finger1_finger_link', 'rg2_gripper_finger1_inner_knuckle_link', 'rg2_gripper_finger1_finger_tip_link', 'rg2_gripper_finger2_finger_link', 'rg2_gripper_finger2_inner_knuckle_link', 'r

### Render

In [18]:
env.init_viewer(viewer_title='Husky',viewer_width=1200,viewer_height=800,
                viewer_hide_menus=True)
env.update_viewer(azimuth=107.08,distance=2.5,elevation=-27,lookat=[0.4,0.3,0.2],
                  VIS_TRANSPARENT=False,VIS_CONTACTPOINT=True,
                  contactwidth=0.1,contactheight=0.1,contactrgba=np.array([1,0,0,1]),
                  VIS_JOINT=True,jointlength=0.5,jointwidth=0.1,jointrgba=[0.2,0.6,0.8,0.6])
# Initial UR5e Pose
init_ur_q = np.array([np.deg2rad(-90), np.deg2rad(-130), np.deg2rad(120), np.deg2rad(100), np.deg2rad(45), np.deg2rad(-90)])

# Change controller gain
for ctrl_name in env.ctrl_names:
    kv = 1
    # env.model.actuator(ctrl_name).gainprm[0] = kv
    # env.model.actuator(ctrl_name).biasprm[2] = -kv

env.reset()
while (env.get_sim_time() < 100.0) and env.is_viewer_alive():
    # Step
    ctrl_husky = 5*np.array([1,1,1,1]) # rotate:[5,-5,5,-5] straight:[5,5,5,5]
    ctrl_wholebody = ctrl_husky.tolist() + init_ur_q.tolist()
    # env.step(ctrl=ctrl_husky, ctrl_idxs=[0,1,2,3])
    env.step(ctrl=ctrl_wholebody, ctrl_idxs=[0,1,2,3,4,5,6,7,8,9])
    # Do render
    if env.loop_every(HZ=30) or (env.tick == 1):
        # Update viewer information
        p_base = env.get_p_body(body_name='base_husky')
        env.update_viewer(lookat=p_base,CALL_MUJOCO_FUNC=False)
        # Render contact information
        PLOT_CONTACT = True
        if PLOT_CONTACT:
            p_contacts,f_contacts,geom1s,geom2s, _,_ = env.get_contact_info()
            for (p_contact,f_contact,geom1,geom2) in zip(p_contacts,f_contacts,geom1s,geom2s):
                f_norm = np.linalg.norm(f_contact)
                f_uv = f_contact / (f_norm+1e-8)
                f_len = 0.3 # f_norm*0.05
                label = '' #'[%s]-[%s]'%(geom1,geom2)
                # env.plot_arrow(p=p_contact,uv=f_uv,r_stem=0.01,len_arrow=f_len,rgba=[1,0,0,0.4],label='')
                # env.plot_arrow(p=p_contact,uv=-f_uv,r_stem=0.01,len_arrow=f_len,rgba=[1,0,0,0.4],label='')
                # env.plot_sphere(p=p_contact,r=0.0001,label=label)
        env.plot_T(p=np.zeros(3),R=np.eye(3,3),PLOT_AXIS=True,axis_len=1.0,axis_width=0.01)
        env.plot_body_T(body_name='base_husky',PLOT_AXIS=True,axis_len=0.1,axis_width=0.01)
        env.plot_body_T(body_name='front_left_wheel_link',PLOT_AXIS=True,axis_len=0.1,axis_width=0.01)
        env.plot_body_T(body_name='front_right_wheel_link',PLOT_AXIS=True,axis_len=0.1,axis_width=0.01)
        env.plot_body_T(body_name='rear_left_wheel_link',PLOT_AXIS=True,axis_len=0.1,axis_width=0.01)
        env.plot_body_T(body_name='rear_right_wheel_link',PLOT_AXIS=True,axis_len=0.1,axis_width=0.01)
        env.plot_T(p=p_base+np.array([0,0,0.5]),R=np.eye(3,3),
                   PLOT_AXIS=False,label='[%.2f]sec'%(env.get_sim_time()))
        env.render()
        
# Close viewer
env.close_viewer()
print ("Time:[%.2f]sec. Done."%(env.get_sim_time()))

Pressed ESC
Quitting.
Time:[0.61]sec. Done.


In [19]:
env.init_viewer(viewer_title='Husky',viewer_width=1200,viewer_height=800,
                viewer_hide_menus=True)
env.update_viewer(azimuth=107.08,distance=2.5,elevation=-27,lookat=[0.4,0.3,0.2],
                  VIS_TRANSPARENT=False,VIS_CONTACTPOINT=True,
                  contactwidth=0.1,contactheight=0.1,contactrgba=np.array([1,0,0,1]),
                  VIS_JOINT=True,jointlength=0.5,jointwidth=0.1,jointrgba=[0.2,0.6,0.8,0.6])

husky_base_pos_list = []
wheel_radius = 0.17775 / 2.0
wheel_base = 0.2854 * 2
# Define the velocity gains
v_max = 5.0  # maximum linear velocity, in m/s
w_max = 50.0  # maximum angular velocity, in rad/s
p_target = np.array([5, -1.5, 0.5])
env.reset()

# # Initial UR5e Pose
# init_ur_q = np.array([np.deg2rad(-90), np.deg2rad(-130), np.deg2rad(120), np.deg2rad(100), np.deg2rad(45), np.deg2rad(-90)])
# env.forward(q=init_ur_q, joint_idxs=[4,5,6,7,8,9])

while (env.get_sim_time() < 100.0) and env.is_viewer_alive():

    # Get the current position of the vehicle from the simulation
    p_base = env.get_p_body('base_husky')
    d_pos = p_target[:2] - p_base[:2]
    # Calculate the distance to the target point
    distance = np.linalg.norm(d_pos)
    if distance < 0.1:
        break

    R_base = r2rpy(env.get_R_body('base_husky'))
    heading = np.arctan2(d_pos[1], d_pos[0]) - R_base[2]
    heading = np.mod(heading + np.pi, 2*np.pi) - np.pi
    
    # compute desired linear and angular velocities
    linear_vel = v_max * np.tanh(distance)
    angular_vel = w_max * np.tanh(heading)
    
    # compute wheel speeds for forward motion
    vl = linear_vel - (angular_vel * wheel_base / 2)
    vr = linear_vel + (angular_vel * wheel_base / 2)

    # Update the simulation state
    ctrl_husky = np.array([vl,vr,vl,vr])
    ctrl_wholebody = ctrl_husky.tolist() + init_ur_q.tolist()
    # env.step(ctrl=ctrl_husky, ctrl_idxs=[0,1,2,3])
    env.step(ctrl=ctrl_wholebody, ctrl_idxs=[0,1,2,3,4,5,6,7,8,9])
    # Do render
    if env.loop_every(HZ=30) or (env.tick == 1):
        # Update viewer information

        env.plot_sphere(p=p_target,r=0.1,rgba=[1,1,1,1],label='')
        env.plot_T(p=p_base+np.array([0,0,0.5]),R=np.eye(3,3),
                   PLOT_AXIS=False,label='[%.2f]sec'%(env.get_sim_time()))
        
        husky_base_pos_list.append(env.get_p_body('base_husky'))
        for p in np.array(husky_base_pos_list)[::5]:
            env.plot_sphere(p=p+np.array([0,0,0.3]), r=0.01) 
        env.render()
    
# Close viewer
env.close_viewer()
print ("Time:[%.2f]sec. Done."%(env.get_sim_time()))



Pressed ESC
Quitting.
Time:[0.71]sec. Done.


In [20]:
## spawn arbitrary objects
obj_cylinder_names = [body_name for body_name in env.body_names
             if body_name is not None and (body_name.startswith("obj_cylinder"))]

n_cylinder_obj = 1 # len(obj_cylinder_names)

# Place objects
env.place_objects(n_obj=n_cylinder_obj, obj_names=obj_cylinder_names, x_range=[p_target[0]+0.7, p_target[0]+1.0], y_range=[p_target[1], p_target[1]+0.1], z_range=[p_target[2], p_target[2]], COLORS=True, VERBOSE=True)

obj_cylinder_01: [ 5.98 -1.47  0.5 ]


In [37]:
env.init_viewer(viewer_title='Husky',viewer_width=1200,viewer_height=800,
                viewer_hide_menus=True)
env.update_viewer(azimuth=107.08,distance=2.5,elevation=-27,lookat=[0.4,0.3,0.2],
                  VIS_TRANSPARENT=False,VIS_CONTACTPOINT=True,
                  contactwidth=0.1,contactheight=0.1,contactrgba=np.array([1,0,0,1]),
                  VIS_JOINT=True,jointlength=0.5,jointwidth=0.1,jointrgba=[0.2,0.6,0.8,0.6])

husky_base_pos_list = []
wheel_radius = 0.17775 / 2.0
wheel_base = 0.2854 * 2
# Define the velocity gains
v_max = 5.0  # maximum linear velocity, in m/s
w_max = 50.0  # maximum angular velocity, in rad/s
p_target = np.array([5, -1.5, 0.5])
STEP = None
env.reset()

while (env.get_sim_time() < 100.0) and env.is_viewer_alive():

    # Get the current position of the vehicle from the simulation
    p_base = env.get_p_body('base_husky')
    d_pos = p_target[:2] - p_base[:2]
    # Calculate the distance to the target point
    distance = np.linalg.norm(d_pos)
    if distance < 0.1:
        STEP = "MANIPULATE"
        # break

    R_base = r2rpy(env.get_R_body('base_husky'))
    heading = np.arctan2(d_pos[1], d_pos[0]) - R_base[2]
    heading = np.mod(heading + np.pi, 2*np.pi) - np.pi
    
    # compute desired linear and angular velocities
    linear_vel = v_max * np.tanh(distance)
    angular_vel = w_max * np.tanh(heading)
    
    # compute wheel speeds for forward motion
    vl = linear_vel - (angular_vel * wheel_base / 2)
    vr = linear_vel + (angular_vel * wheel_base / 2)

    # Update the simulation state
    if STEP == "MANIPULATE":
        ctrl_husky = np.zeros(4)
        ctrl_ur = q_ik
        ctrl_wholebody = ctrl_husky.tolist() + ctrl_ur.tolist()
        # env.step(ctrl=ctrl_husky, ctrl_idxs=[0,1,2,3])
        env.step(ctrl=ctrl_wholebody, ctrl_idxs=[0,1,2,3,4,5,6,7,8,9])
    else:
        ctrl_husky = np.array([vl,vr,vl,vr])
        ctrl_wholebody = ctrl_husky.tolist() + init_ur_q.tolist()
        env.step(ctrl=ctrl_wholebody, ctrl_idxs=[0,1,2,3,4,5,6,7,8,9])
    # Do render
    if env.loop_every(HZ=30) or (env.tick == 1):
        # Update viewer information

        env.plot_sphere(p=p_target,r=0.1,rgba=[1,1,1,1],label='')
        env.plot_T(p=p_base+np.array([0,0,0.5]),R=np.eye(3,3),
                   PLOT_AXIS=False,label='[%.2f]sec'%(env.get_sim_time()))
        
        husky_base_pos_list.append(env.get_p_body('base_husky'))
        for p in np.array(husky_base_pos_list)[::5]:
            env.plot_sphere(p=p+np.array([0,0,0.3]), r=0.01) 
        env.render()
    
# Close viewer
env.close_viewer()
print ("Time:[%.2f]sec. Done."%(env.get_sim_time()))



Pressed ESC
Quitting.
Time:[15.39]sec. Done.


In [22]:
# Target object
for idx, obj_name in enumerate(obj_cylinder_names):
    print(env.get_p_body(obj_name))

[ 5.98 -1.47  0.22]
[-1.56  0.05 -0.01]
[-1.64  0.08  0.01]
[-1.76  0.04 -0.01]
[-1.6  -0.33  0.02]
[-1.87  0.4  -0.01]
[-1.84  0.18 -0.01]
[-1.86 -0.07  0.  ]


In [27]:
env.idxs_jacobian

[6, 7, 8, 9, 10, 11, 12]

In [34]:
env.init_viewer(viewer_title='UR5e with RG2 gripper',viewer_width=1200,viewer_height=800,
                viewer_hide_menus=True)
env.update_viewer(azimuth=0.0,distance=3.5,elevation=-60,lookat=[0.4,0.05,0.36],
                  VIS_TRANSPARENT=False,VIS_CONTACTPOINT=True,
                  contactwidth=0.05,contactheight=0.05,contactrgba=np.array([1,0,0,1]),
                  VIS_JOINT=True,jointlength=0.5,jointwidth=0.1,jointrgba=[0.2,0.6,0.8,0.6])
env.reset()

p_trgt = env.get_p_body('obj_cylinder_01')
R_trgt = env.get_R_body('obj_cylinder_01')
q_ik = env.solve_ik('obj_cylinder_01',p_trgt,R_trgt,IK_P=True,IK_R=True,q_init=init_ur_q,rev_joint_idxs=[4,5,6,7,8,9],
                 RESET=False,DO_RENDER=True,render_every=1,th=1*np.pi/180.0,err_th=1e-6)

# Close viewer
env.close_viewer()
print ("Done.")


Done.


In [35]:
q_ik

array([-0.45, -0.65,  0.6 ,  1.75,  0.79, -1.57])

array([0. , 0. , 0.1])